In [ ]:
import json
from tqdm import tqdm

## Download the dataset

In [ ]:
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
!wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip

--2022-11-21 06:33:40--  https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3950125 (3.8M) [application/zip]
Saving to: ‘e2eqa-train+public_test-v1.zip’

e2eqa-train+public_ 100%[===================>]   3.77M  3.07MB/s    in 1.2s    

2022-11-21 06:33:42 (3.07 MB/s) - ‘e2eqa-train+public_test-v1.zip’ saved [3950125/3950125]

--2022-11-21 06:33:42--  https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417016977 (398M) [application/zip]
Saving to: ‘wikipedia_20220620_cleaned.zip’

wikipedia_20220620_

In [ ]:
# Stopwords
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt

--2022-11-21 06:34:08--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords-dash.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0.001s  

2022-11-21 06:34:08 (13.5 MB/s) - ‘vietnamese-stopwords-dash.txt’ saved [20475/20475]



In [ ]:
#unzip the zaloAI data

!unzip -q /content/e2eqa-train+public_test-v1.zip
!unzip -q /content/wikipedia_20220620_cleaned.zip

In [ ]:
data_file = open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json')
data = json.load(data_file)

In [ ]:
# check out the data
data

In [ ]:
# train data format that we need for BERT


"""train_data = [
    {
        "context": "Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.",
        "qas": [
            {
                "id": "00001",
                "is_impossible": False,
                "question": "Who is the author of the Mistborn series?",
                "answers": [
                    {
                        "text": "Brandon Sanderson",
                        "answer_start": 71,
                    }
                ],
            }
        ],
    },
    {
        "context": "The first series, published between 2006 and 2008, consists of The Final Empire,"
                   "The Well of Ascension, and The Hero of Ages.",
        "qas": [
            {
                "id": "00002",
                "is_impossible": False,
                "question": "When was the series published?",
                "answers": [
                    {
                        "text": "between 2006 and 2008",
                        "answer_start": 28,
                    }
                ],
            },
            {
                "id": "00003",
                "is_impossible": False,
                "question": "What are the three books in the series?",
                "answers": [
                    {
                        "text": "The Final Empire, The Well of Ascension, and The Hero of Ages",
                        "answer_start": 63,
                    }
                ],
            },
            {
                "id": "00004",
                "is_impossible": True,
                "question": "Who is the main character in the series?",
                "answers": [],
            },
        ],
    },
]"""


In [ ]:

n = len(data["data"])
n 

20857

In [ ]:
# create a list to store data
training= []

In [ ]:
#take the data section of the original data into Datapack 
Datapack= data['data']
Datapack[1]

{'id': 'c926e7b0717202618a10dd907d4b4c39',
 'question': 'Đất nước nào không có quân đội',
 'title': '',
 'text': 'có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
 'short_candidate_start': 53,
 'short_candidate': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
 'answer': 'wiki/Danh_sách_quốc_gia_không_có_lực_lượng_vũ_trang',
 'category': 'FULL_ANNOTATION',
 'is_long_answer': True}

In [ ]:
# create QAS key 
def defineQAS (datapack): 
  qas = list()
  packet= dict()

  packet['id'] = datapack['id'] 
  packet['is_impossible'] = False 
  packet['question'] = datapack['question']


  answerPack = list()
  SmallAnswerPack= dict()

  SmallAnswerPack['text'] = datapack['short_candidate']
  SmallAnswerPack['answer_start'] = datapack['short_candidate_start']

  answerPack.append(SmallAnswerPack)

  packet['answers'] = answerPack

  qas.append(packet) 



  return qas




### complete the dataset

In [ ]:
# define each packet as a dictionary
Datapack= data['data']
i = 0 
for i in range(0, n):
  # remove all the FALSE_LONG_ANSWER and PARTIAL_ANNOTATION
  if Datapack[i]['category'] == 'FALSE_LONG_ANSWER':
    continue
  elif Datapack[i]['category'] == 'PARTIAL_ANNOTATION': 
    continue

  packet= dict()


  packet['context'] = Datapack[i]['text']
  packet['qas'] = defineQAS(Datapack[i])






  training.append(packet.copy())











### Now we got the data format for BERT let's create train.json and test.json 

In [ ]:
# split into train and test dataset

from sklearn.model_selection import train_test_split

X = training  # independant features
				# dependant variable

# Choose your test size to split between training and testing sets:
train, test= train_test_split(X, test_size=0.25, random_state=42)

In [ ]:
json_object = json.dumps(train, indent=4, ensure_ascii=False)

In [ ]:
json_object = json.dumps(test, indent=4, ensure_ascii=False)

In [ ]:
with open("train.json", "w", encoding='utf-8') as result:
    result.write(json_object)

In [ ]:
with open("test.json", "w", encoding='utf-8') as result:
    result.write(json_object)

In [ ]:
data_file = open('train.json')
train = json.load(data_file)
train[4]

{'context': '- Chùa Linh Sơn Tiên Thạch , tên gọi khác của chùa Bà Đen nằm trên núi Bà Đen , tỉnh Tây Ninh .',
 'qas': [{'id': 'e50c68585e0d5587a33f0c17ecf2dc98',
   'is_impossible': False,
   'question': 'núi bà đen nằm ở tỉnh nào',
   'answers': [{'text': 'tỉnh Tây Ninh', 'answer_start': 80}]}]}

In [ ]:
data_file = open('test.json')
train = json.load(data_file)
train[4]

{'context': '- Chùa Linh Sơn Tiên Thạch , tên gọi khác của chùa Bà Đen nằm trên núi Bà Đen , tỉnh Tây Ninh .',
 'qas': [{'id': 'e50c68585e0d5587a33f0c17ecf2dc98',
   'is_impossible': False,
   'question': 'núi bà đen nằm ở tỉnh nào',
   'answers': [{'text': 'tỉnh Tây Ninh', 'answer_start': 80}]}]}

In [ ]:
questions_file = open('e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json')
questions = json.load(questions_file)
questions

In [ ]:
# evaluation that we need 

''' eval_data = [
    {
        "context": "The series primarily takes place in a region called the Final Empire "
                   "on a world called Scadrial, where the sun and sky are red, vegetation is brown, "
                   "and the ground is constantly being covered under black volcanic ashfalls.",
        "qas": [
            {
                "id": "00001",
                "is_impossible": False,
                "question": "Where does the series take place?",
                "answers": [
                    {
                        "text": "region called the Final Empire",
                        "answer_start": 38,
                    },
                    {
                        "text": "world called Scadrial",
                        "answer_start": 74,
                    },
                ],
            }
        ],
    },
    {
        "context": "\"Mistings\" have only one of the many Allomantic powers, while \"Mistborns\" have all the powers.",
        "qas": [
            {
                "id": "00002",
                "is_impossible": False,
                "question": "How many powers does a Misting possess?",
                "answers": [
                    {
                        "text": "one",
                        "answer_start": 21,
                    }
                ],
            },
            {
                "id": "00003",
                "is_impossible": True,
                "question": "What are Allomantic powers?",
                "answers": [],
            },
        ],
    },
]'''